In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
class node(object):
    
    def __init__(self,state):
        self.state=state
        self.inbound={}
        self.outbound={}
        self.cost=0
        
    def __str__(self):
        string="State: "+str(self.state)
        return string

In [3]:
n_sensitivity=10
W_sensitivity=40

c_rand=[10,8,5,8,7,3,4,4,2,4]
w_rand=[8,6,10,9,9,2,8,10,7,4]

c_w_asc=[3,4,8,2,10,4,8,7,5,4]
w_w_asc=[2,4,6,7,8,8,9,9,10,10]

c_w_des=c_w_asc[::-1]
w_w_des=w_w_asc[::-1]

c_c_asc=[2,3,4,4,4,5,7,8,8,10]
w_c_asc=[7,2,4,8,10,10,9,6,9,8]

c_c_des=c_c_asc[::-1]
w_c_des=w_c_asc[::-1]

c_r_asc=[2,4,4,5,7,8,4,10,8,3]
w_r_asc=[7,10,8,10,9,9,4,8,6,2]

c_r_des=c_r_asc[::-1]
w_r_des=w_r_asc[::-1]

c_testset=[c_rand,c_w_asc,c_w_des,c_c_asc,c_c_des,c_r_asc,c_r_des]
w_testset=[w_rand,w_w_asc,w_w_des,w_c_asc,w_c_des,w_r_asc,w_r_des]

In [4]:
# n_sensitivity=20
# W_sensitivity=80

# c_rand=[7,7,2,10,10,2,3,10,7,3,5,8,9,10,4,3,9,10,9,1]
# w_rand=[6,9,8,3,8,6,10,5,9,4,6,8,3,7,10,8,4,10,6,10]

# c_w_asc=[10,9,3,9,10,7,2,5,9,10,2,10,8,3,7,7,3,4,10,1]
# w_w_asc=[3,3,4,4,5,6,6,6,6,7,8,8,8,8,9,9,10,10,10,10]

# c_w_des=c_w_asc[::-1]
# w_w_des=w_w_asc[::-1]

# c_c_asc=[1,2,2,3,3,3,4,5,7,7,7,8,9,9,9,10,10,10,10,10]
# w_c_asc=[10,8,6,10,8,4,10,6,9,9,6,8,6,4,3,10,8,7,5,3]

# c_c_des=c_c_asc[::-1]
# w_c_des=w_c_asc[::-1]

# c_r_asc=[1,2,3,2,3,4,3,7,7,5,8,10,7,10,10,9,10,9,9,10]
# w_r_asc=[10,8,10,6,8,10,4,9,9,6,8,10,6,8,7,6,5,4,3,3]

# c_r_des=c_r_asc[::-1]
# w_r_des=w_r_asc[::-1]

# c_testset=[c_rand,c_w_asc,c_w_des,c_c_asc,c_c_des,c_r_asc,c_r_des]
# w_testset=[w_rand,w_w_asc,w_w_des,w_c_asc,w_c_des,w_r_asc,w_r_des]

In [5]:
c_test=[x for _,x in sorted(zip(w_rand,c_rand))][::-1]
w_test=np.sort(w_rand)[::-1]
print(c_test)
print(w_test)
for i in range(len(w_rand)):
    print("c: {} w: {}".format(c_w_asc[i],w_w_asc[i]))

[5, 4, 8, 7, 10, 4, 2, 8, 4, 3]
[10 10  9  9  8  8  7  6  4  2]
c: 3 w: 2
c: 4 w: 4
c: 8 w: 6
c: 2 w: 7
c: 10 w: 8
c: 4 w: 8
c: 8 w: 9
c: 7 w: 9
c: 5 w: 10
c: 4 w: 10


In [6]:
def Algorithm_1(n,cvals,wvals,W):
    
    #All layers is the tree containing each layer
    #All_layers keys are the layer numbers
    #Make root node before beginning
    all_layers={}
    all_layers[0]={}
    all_layers[0][0]=node(0)
    
    #Iterate through each layer
    for x in range(1,n):
        #Create next layer
        currLayer=all_layers[x-1]
        all_layers[x]={}
        nextLayer=all_layers[x]
        
        #Initialize layer variables
        #Saves having to compute this for each node in the layer
        w_current=wvals[x-1]
        c_current=cvals[x-1]
        
        #Begin with the 0 decision: deciding not to take the item
        #Node weight remains the same, so create new node in the subsequent layer
        #Direct arcs to new node
        for nodeInstance in currLayer.values():
            new_node=node(nodeInstance.state)
            nextLayer[nodeInstance.state]=new_node
            nodeInstance.outbound[0]=new_node
            new_node.inbound[0]=nodeInstance
        
        #Next do the 1 decision: deciding to take the item
        for nodeInstance in currLayer.values():
            newstate=nodeInstance.state+w_current
            #Feasability check
            if newstate<=W:
                #Check if the new state already exists in the subsequent layer
                if nextLayer.get(newstate):
                    #Add arc into node inbound and outbound
                    nodeInstance.outbound[c_current]=nextLayer[newstate]
                    nextLayer[newstate].inbound[c_current]=nodeInstance
                else:
                    #Create new node
                    new_node=node(newstate)
                    nextLayer[newstate]=new_node
                    #Add arc to node inbound and outbound
                    nodeInstance.outbound[c_current]=new_node
                    new_node.inbound[c_current]=nodeInstance
                        
    #Changed from W because Rachel said so
    terminal_node=node(-1)
    all_layers[n]={}
    all_layers[n][-1]=terminal_node
    
    w_final=wvals[n-1]
    c_final=cvals[n-1]
    
    #Swap arc dictionary keys and values
    #This method still leads to probelms in the inbound nodes as you can have multiple inbound arc costs
    #But hey, it works for outbound
    
    #Final layer 0 decision
    for nodeInstance in all_layers[n-1].values():
        
        nodeInstance.outbound[0]=terminal_node
        terminal_node.inbound[0]=nodeInstance
    
    #Final layer 1 decision
    for nodeInstance in all_layers[n-1].values():
        
        if nodeInstance.state+w_final<=W:
            nodeInstance.outbound[c_final]=terminal_node
            terminal_node.inbound[c_final]=nodeInstance
            
    return all_layers

In [7]:
def Algorithm_2(reduced_layers,n):
    #Final reduction algorithm here we go
    #Starting from the second last layer moving up
    for x in range(n-1,0,-1):
        to_remove={}
        tail_list={}
        for nodeInstance in reduced_layers[x].values():
            #Add ending combination if not already in the tail list
            current_tuple=tuple(nodeInstance.outbound.items())
            
            #If ending combination exists
            if tail_list.get(current_tuple):
                to_remove[nodeInstance.state]=nodeInstance
                #Redirect arcs
                for arcCost, comingFrom in nodeInstance.inbound.items():
                    tail_list[current_tuple].inbound[arcCost]=comingFrom
                    comingFrom.outbound[arcCost]=tail_list[current_tuple]
                    
                    #Remove arcs from comingFrom node
                    if comingFrom.outbound.get(nodeInstance):
                        del(comingFrom.outbound[nodeInstance])
            
            #If ending combination does not exist
            else:
                tail_list[current_tuple]=nodeInstance
                        
        #Remove all duplicate nodes            
        for remove_node in to_remove.values():
            del(reduced_layers[x][remove_node.state])
            remove_node=None
        
    return reduced_layers

In [8]:
def Longest_Path(reduced_layers,cvals):
    #Initiate final score and trail list
    n=len(reduced_layers)-1
    final_score=0
    trail=[0]*(len(reduced_layers)-1)
    
    #Iterate through every node and incoming arc, update highest cost found
    for layer in reduced_layers.values():
        for nodeInstance in layer.values():
            for arcCost, goingTo in nodeInstance.outbound.items():
                #Only update arcs where cost > 0
                if (nodeInstance.cost+arcCost)>goingTo.cost:
                    goingTo.cost=nodeInstance.cost+arcCost
                    
    #In the last layer of nodes find the node with the largest cost and set it as the final score
    for nodeInstance in reduced_layers[n].values():
        final_score=nodeInstance.cost
        break
    
    #Initialize traceback score (optima value), and track node progress
    traceback=final_score
    current_node=reduced_layers[n][-1]
    
    for x in range(n,0,-1):
        
        #Iterate through all layers to find current node
        #Possibly reduce solve time even further by only tracing back through current_node
        #Keep track of when current_node is contained in the root layer
        #If so, you've reached the top
        
        previous_layer=reduced_layers.get(x-1)
        c_current=cvals[x-1]
        target=traceback-c_current
        
        #Iterate through each node
        for nodeInstance in previous_layer.values():
            
            #If take item is an option, do it
            if nodeInstance.cost==target:
                #Ensure that the node is along the same trail as lower levels
                if nodeInstance.outbound.get(c_current) is current_node:
                    #Update trail, current node, and traceback
                    trail[x-1]=1
                    current_node=nodeInstance
                    traceback-=c_current
                    break
            
            #If don't take is an option, do it as well
            elif nodeInstance.cost==traceback:
                #Update current node
                if nodeInstance.outbound.get(0) is current_node:
                    current_node=nodeInstance
                    break
    
            
    return final_score, trail

In [9]:
def Max_Width(all_layers):
    
    max_width=0
    node_count=0
    
    for layer in all_layers.values():
        no_of_nodes=len(layer)
        node_count+=no_of_nodes
        if no_of_nodes>max_width:
            max_width=no_of_nodes
            
    return max_width, node_count

In [10]:
def Arc_Count(all_layers):
    
    arc_count=0
    
    for layer in all_layers.values():
        for nodeInstance in layer.values():
            arc_count+=len(nodeInstance.outbound)
    
    return arc_count

In [11]:
def Order_Testing():
    #Initialize testing varibales
    testset=[n_sensitivity,W_sensitivity]

    for nono in range(0,7):
        #Build BDD tree and solve
        start1=time.perf_counter()
        all_layers=Algorithm_1(testset[0],c_testset[nono],w_testset[nono],testset[1])
        [full_finalScore,full_trail]=Longest_Path(all_layers,c_testset[nono])
        [all_width, all_nodeCount]=Max_Width(all_layers)
        all_arcCount=Arc_Count(all_layers)
        end1=time.perf_counter()

        #Reduce BDD and solve
        start2=time.perf_counter()
        reduced_layers=Algorithm_2(all_layers,testset[0])
        [reduced_finalScore,reduced_trail]=Longest_Path(reduced_layers,c_testset[nono])
        [reduced_width, reduced_nodeCount]=Max_Width(reduced_layers)
        reduced_arcCount=Arc_Count(reduced_layers)
        end2=time.perf_counter()

        print("Test no. {}\n".format(nono+1))
        print("Full BDD number of nodes: {}".format(all_nodeCount))
        print("Reduced BDD number of nodes: {}".format(reduced_nodeCount))
        print("Full BDD number of arcs: {}".format(all_arcCount))
        print("Reduced BDD number of arcs: {}".format(reduced_arcCount))
        print("Full BDD max width: {}".format(all_width))
        print("Reduced BDD max width: {}\n".format(reduced_width))
        
    return None

In [12]:
print("Testing Order \n")
print("1: Random")
print("2: w ascending")
print("3: w descending")
print("4: c ascending")
print("5: c descending")
print("6: ratio ascending")
print("7: ratio descending\n\n")

Order_Testing()

Testing Order 

1: Random
2: w ascending
3: w descending
4: c ascending
5: c descending
6: ratio ascending
7: ratio descending


Test no. 1

Full BDD number of nodes: 189
Reduced BDD number of nodes: 78
Full BDD number of arcs: 350
Reduced BDD number of arcs: 144
Full BDD max width: 37
Reduced BDD max width: 17

Test no. 2

Full BDD number of nodes: 194
Reduced BDD number of nodes: 52
Full BDD number of arcs: 356
Reduced BDD number of arcs: 98
Full BDD max width: 38
Reduced BDD max width: 10

Test no. 3

Full BDD number of nodes: 152
Reduced BDD number of nodes: 52
Full BDD number of arcs: 283
Reduced BDD number of arcs: 91
Full BDD max width: 37
Reduced BDD max width: 10

Test no. 4

Full BDD number of nodes: 207
Reduced BDD number of nodes: 72
Full BDD number of arcs: 382
Reduced BDD number of arcs: 134
Full BDD max width: 38
Reduced BDD max width: 17

Test no. 5

Full BDD number of nodes: 176
Reduced BDD number of nodes: 72
Full BDD number of arcs: 327
Reduced BDD number of arcs: 12